# NER for Ukrainian

The task is to implement the recognition of four entity types: person ("ПЕРС"), location ("ЛОК"), organization ("ОРГ"), and other ("РІЗН"). The non-entity words will be labeled with "--".

## 1. Process the data

In [1]:
PATH = "ner-uk/"

In [2]:
def position(idx, range_len):
    if idx == 0:
        return "start"
    elif idx == range_len - 1:
        return "end"
    else:
        return "middle"

def read_tokens(filename):
    """Read tokens and positions of tokens from a *.tok.txt file"""
    tokens = []
    pos = 0
    with open(filename, "r") as f:
        text = f.read().split("\n")
        for line in text:
            if len(line) == 0:
                pos += 1
            else:
                line = line.split(" ")
                for i in range(len(line)):
                    token = line[i]
                    tokens.append((token, pos, pos + len(token), position(i, len(line))))
                    pos += len(token) + 1
    return tokens

In [3]:
def read_annotations(filename):
    """Read annotations and positions of annotations from a *.tok.ann file"""
    anno = []
    with open(filename, "r") as f:
        for line in f.readlines():
            annotations = line.split()
            anno.append((annotations[1], int(annotations[2]), int(annotations[3])))
    return anno

In [4]:
tokens = read_tokens(PATH + "data/A_alumni.krok.edu.ua_Prokopenko_Vidrodzhennia_velotreku(5).tok.txt")
anno = read_annotations(PATH + "data/A_alumni.krok.edu.ua_Prokopenko_Vidrodzhennia_velotreku(5).tok.ann")

for token in tokens[:30]:
    print(token)

print("")
for ann in anno[:5]:
    print(ann)

('Історія', 0, 7, 'start')
('змін', 8, 12, 'middle')
('.', 13, 14, 'end')
('Спільними', 16, 25, 'start')
('зусиллями', 26, 35, 'middle')
('влада', 36, 41, 'middle')
('та', 42, 44, 'middle')
('громадськість', 45, 58, 'middle')
('врятували', 59, 68, 'middle')
('й', 69, 70, 'middle')
('повертають', 71, 81, 'middle')
('до', 82, 84, 'middle')
('життя', 85, 90, 'middle')
('Київський', 91, 100, 'middle')
('велотрек', 101, 109, 'end')
('Київський', 110, 119, 'start')
('велотрек', 120, 128, 'middle')
('«', 129, 130, 'middle')
('Авангард', 131, 139, 'middle')
('»', 140, 141, 'middle')
('по', 142, 144, 'middle')
('вул', 145, 148, 'middle')
('.', 149, 150, 'middle')
('Богдана', 151, 158, 'middle')
('Хмельницького', 159, 172, 'middle')
(',', 173, 174, 'middle')
('58-А', 175, 179, 'middle')
(',', 180, 181, 'middle')
('що', 182, 184, 'middle')
('збудований', 185, 195, 'middle')

('ОРГ', 91, 109)
('ОРГ', 110, 141)
('ЛОК', 145, 179)
('ОРГ', 513, 548)
('ПЕРС', 549, 560)


In [5]:
def extract_labels(anno, tokens):
    """Extract labels for tokens"""
    labels = []
    ann_id = 0
    for token in tokens:
        if ann_id < len(anno):
            label, beg, end = anno[ann_id]
            if token[1] < beg:
                labels.append("--")
            else:
                if token[1] == beg:
                    labels.append("B-" + label)
                else:
                    labels.append("I-" + label)
                if token[2] == end:
                    ann_id += 1
        else:
            labels.append("--")    
    return labels

labels = extract_labels(anno, tokens)

In [6]:
for i, j in zip(tokens, labels):
    print(i[0], j)

Історія --
змін --
. --
Спільними --
зусиллями --
влада --
та --
громадськість --
врятували --
й --
повертають --
до --
життя --
Київський B-ОРГ
велотрек I-ОРГ
Київський B-ОРГ
велотрек I-ОРГ
« I-ОРГ
Авангард I-ОРГ
» I-ОРГ
по --
вул B-ЛОК
. I-ЛОК
Богдана I-ЛОК
Хмельницького I-ЛОК
, I-ЛОК
58-А I-ЛОК
, --
що --
збудований --
у --
1913 --
році --
за --
ініціативи --
та --
кошти --
киян --
, --
відновлюється --
так --
само --
— --
силами --
громади --
і --
без --
фінансування --
з --
бюджету --
. --
А --
за --
відчутної --
підтримки --
влади --
реконструкція --
набирає --
обертів --
. --
« --
Ще --
недавно --
велотрек --
існував --
тільки --
у --
мріях --
ентузіастів --
велоруху --
, --
а --
вже --
зараз --
він --
стрімко --
набирає --
реалістичних --
контурів --
, --
— --
радіє --
голова --
Шевченківської B-ОРГ
райдержадміністрації I-ОРГ
Олег B-ПЕРС
Гаряга I-ПЕРС
. --
— --
Ми --
сподіваємося --
, --
що --
вже --
за --
півтора-два --
місяці --
на --
велотреку --
зможуть --
тренуватися --
сп

## 2. Prepare the dev/test data

In [7]:
dev_test = {"dev": [], "test": []}
category = ""
with open(PATH + "doc/dev-test-split.txt", "r") as f:
    for line in f.readlines():
        line = line.strip()
        if line in ["DEV", "TEST"]:
            category = line.lower()
        elif len(line) == 0:
            continue
        else:
            dev_test[category].append(line)

print(len(dev_test["dev"]), len(dev_test["test"]))

156 73


In [8]:
train_tokens, test_tokens, train_labels, test_labels = [], [], [], []

for filename in dev_test["dev"]:
    try:
        tokens = read_tokens(PATH + "data/" + filename + ".txt")
        train_tokens += [(token[0], token[3]) for token in tokens]
        train_labels += extract_labels(read_annotations(PATH + "data/" + filename + ".ann"), tokens)
    except:
        print(filename)

for filename in dev_test["test"]:
    try:
        tokens = read_tokens(PATH + "data/" + filename + ".txt")
        test_tokens += [(token[0], token[3]) for token in tokens]
        test_labels += extract_labels(read_annotations(PATH + "data/" + filename + ".ann"), tokens)
    except:
        print(filename)

A_Kiyanovska_2_Ya_ne_poet_2014(4).tok
A_MOU_Pratsivnyk_ZS_Ukrayiny_Oleh_Vernyayev_zdobuvaye_2015(4).tok
A_Prokopenko_Vidrodzhennia_velotreku(5).tok
A_prokovel_Kovelchanka_Tetiana_Kob_vyborola_Kubok_Ukrainy_z_boksu_2015(4).tok
A_gukr_5_finansovykh_variantiv_yaki_mozhut_buty_korysni_dla_pensioneriv_1_2013(5).tok
A_Kiyanovska_1_Ya_ne_poet_2014(4).tok


In [9]:
print(len(train_tokens), len(test_tokens), len(train_labels), len(test_labels))

143939 70063 143939 70063


In [10]:
for i, j in zip(train_tokens[:100], train_labels[:100]):
    print(i, j),

('На', 'start') --
('довірливих', 'middle') --
('кіровоградців', 'middle') --
('полюють', 'middle') --
('шахраї', 'middle') --
('та', 'middle') --
('фірми-посередники', 'middle') --
(',', 'middle') --
('які', 'middle') --
('за', 'middle') --
('1000', 'middle') --
('грн', 'middle') --
('.', 'middle') --
('готові', 'middle') --
('«', 'middle') --
('виготовити', 'middle') --
('»', 'middle') --
('біометричний', 'middle') --
('паспорт', 'middle') --
(',', 'middle') --
('який', 'middle') --
('коштує', 'middle') --
('518', 'middle') --
('грн', 'middle') --
('.', 'end') --
('Із', 'start') --
('запровадженням', 'middle') --
('біометричних', 'middle') --
('паспортів', 'middle') --
('активізувалися', 'middle') --
('шахраї', 'middle') --
('та', 'middle') --
('фірми-посередники', 'middle') --
(',', 'middle') --
('які', 'middle') --
('пропонують', 'middle') --
('«', 'middle') --
('прискорити', 'middle') --
('»', 'middle') --
('оформлення', 'middle') --
('біометричного', 'middle') --
('паспорта', 'mi

### TODO

1. Build a baseline solution
2. Extract features and train Logistic Regression; check top features for each class
3. Try smarter features: POS, sentence boundaries, lemmas, etc.


## Baseline solution

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [12]:
def majority_class(tokens):
    labels = []
    for i in range(len(tokens)):
        word = tokens[i]
        if word[0].istitle() and not word[1] == "start":
            if i > 0 and tokens[i-1][0].istitle():
                labels.append("I-ПЕРС")
            else:
                labels.append("B-ПЕРС")
        else:
            labels.append("--")
    return labels

predicted = majority_class(test_tokens)
print(classification_report(test_labels, predicted))

/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

          --       0.91      0.99      0.95     61837
       B-ЛОК       0.00      0.00      0.00       414
       B-ОРГ       0.00      0.00      0.00       230
      B-ПЕРС       0.41      0.76      0.53      1190
      B-РІЗН       0.00      0.00      0.00       178
       I-ЛОК       0.00      0.00      0.00      1071
       I-ОРГ       0.00      0.00      0.00      1958
      I-ПЕРС       0.50      0.11      0.18      2808
      I-РІЗН       0.00      0.00      0.00       377

   micro avg       0.90      0.90      0.90     70063
   macro avg       0.20      0.21      0.18     70063
weighted avg       0.83      0.90      0.86     70063



## Better solution

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [14]:
print(morph.parse("Київського")[0].tag.POS)
print(morph.parse("Київського")[0].normal_form)

ADJF
київський


In [15]:
# Feature extraction

def get_shape(word):
    if word.istitle():
        return "title"
    elif word.isupper():
        return "upper"
    elif word.isdigit():
        return "digit"
    elif word.islower():
        return "lower"
    else:
        return "other"

def feature_extractor(tokens, ind):
    """
    Collect features for the INDth token in SENTENCE.
    """
    token = tokens[ind]
    features = dict()
    features["shape"] = get_shape(token[0])
    features["is_not_punct"] = token[0].isalnum()
    features["pos"] = str(morph.parse(token[0])[0].tag.POS)
    features["lemma"] = morph.parse(token[0])[0].normal_form
    features["position"] = token[1]
    features["word-1"] = morph.parse(tokens[ind-1][0])[0].normal_form if token[1] != "start" else "<S>"
    features["word-2"] = morph.parse(tokens[ind-2][0])[0].normal_form if ind > 1 and tokens[ind-1][1] != "start" else "<S>"
    if token[1] != "start":
        features["word-1-shape"] = get_shape(tokens[ind-1][0])
        features["word-1-not_punct"] = tokens[ind-1][0].isalnum()
    features["word+1"] = morph.parse(tokens[ind+1][0])[0].normal_form if token[1] != "end" else "</S>"
    features["word+2"] = morph.parse(tokens[ind+2][0])[0].normal_form if ind < (len(tokens) - 2) and tokens[ind+1][1] != "end" else "</S>"
    if token[1] != "end":
        features["word+1-shape"] = get_shape(tokens[ind+1][0])
        features["word+1-not_punct"] = tokens[ind+1][0].isalnum()
    return features

In [16]:
train_features = []

for i in range(len(train_tokens)):
    train_features.append(feature_extractor(train_tokens, i))

print(len(train_features))

143939


In [17]:
print(train_tokens[152])
print(train_features[152])

('ДМС', 'middle')
{'shape': 'upper', 'is_not_punct': True, 'pos': 'None', 'lemma': 'дмс', 'position': 'middle', 'word-1': 'підрозділ', 'word-2': 'у', 'word-1-shape': 'lower', 'word-1-not_punct': True, 'word+1': 'цей', 'word+2': 'процедура', 'word+1-shape': 'lower', 'word+1-not_punct': True}


In [18]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("\nTotal number of features: ", len(vec.get_feature_names()))


Total number of features:  92955


In [19]:
train_features_vectorized = vec.transform(train_features)

print(len(train_features_vectorized.toarray()))

143939


In [20]:
lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial", max_iter=1300, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 1201 epochs took 343 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.7min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1300, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=1, warm_start=False)

In [21]:
test_features = []

for i in range(len(test_tokens)):
    test_features.append(feature_extractor(test_tokens, i))

predicted = lrc.predict(vec.transform(test_features))
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

          --       0.92      1.00      0.95     61837
       B-ЛОК       0.66      0.61      0.63       414
       B-ОРГ       0.63      0.27      0.38       230
      B-ПЕРС       0.70      0.77      0.73      1190
      B-РІЗН       0.43      0.43      0.43       178
       I-ЛОК       0.51      0.03      0.05      1071
       I-ОРГ       0.81      0.05      0.10      1958
      I-ПЕРС       0.57      0.09      0.16      2808
      I-РІЗН       0.49      0.09      0.15       377

   micro avg       0.90      0.90      0.90     70063
   macro avg       0.63      0.37      0.40     70063
weighted avg       0.88      0.90      0.87     70063



In [22]:
import numpy as np

def top_features(vectorizer, clf, n):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vec.get_feature_names()
    for i, class_label in enumerate(clf.classes_):
        top = np.argsort(clf.coef_[i])
        reversed_top = top[::-1]
        print("%s: %s\n" % (class_label,
              " ".join(feature_names[j] for j in reversed_top[:n])))


In [23]:
top_features(vec, lrc, 20)

--: shape=lower lemma=- lemma=війна lemma=, lemma=– word-1=- lemma=« word-1=… word-1=... lemma=— lemma=йога lemma=скарб lemma=інтернет lemma=петрів lemma=: word-2=... word+1=ст word-1=дора lemma=.. word+1=бог

B-ЛОК: lemma=вул lemma=вулиця lemma=рондо lemma=кий lemma=польща lemma=парка lemma=україна word+1=район word-1=на lemma=америка word-1=у pos=NOUN word-1=під lemma=росія pos=ADJF lemma=львов word-1=в lemma=київ word-1=, lemma=оперний

B-ОРГ: lemma=академія shape=upper lemma=міністерство word-1=голова word+1=служба lemma=рад lemma=східside lemma=церква lemma=інститут lemma=адміністрація lemma=обласний word+1=рад lemma=евромайдан lemma=облрада lemma=міграційний word-2=клуб word-1=представник lemma=океан lemma=міськрада word-1=“

B-ПЕРС: lemma=яся shape=title lemma=дора lemma=данка lemma=ілія lemma=вероніка lemma=ярок word-1=, lemma=роман lemma=зіркий lemma=мішель word+1=макарій lemma=інга lemma=божий pos=NOUN lemma=наталочка lemma=гоголь lemma=мирон lemma=бог word-2=<S>

B-РІЗН: wor